# Natural Language Processing Application: Sentimental Analysis on Steam Reviews (Possibly?)

## Team

* Gabriel Aracena
* Joshua Canode
* Aaron Galicia

### Project Description

A key area of knowledge in data analytics is the ability to extract meaning from text. This assignment provides the foundational skills in this area by detecting whether a text conveys a positive or negative message.

Analyze the sentiment (e.g., negative, neutral, positive) conveyed in a large body (corpus) of texts using the NLTK package in Python. Complete the steps below. Then, write a comprehensive technical report as a Python Jupyter notebook to include all code, code comments, all outputs, plots, and analysis. Make sure the project documentation contains a) Problem statement, b) Algorithm of the solution, c) Analysis of the findings, and d) References.

## Abstract

TODO

### Data Preparation:

TODO

### ANN Model Building:

TODO


### Training the ANN:



### Evaluation:


## Model Architecture


## Interpretation and Conclusion



In [215]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import nltk


Importing the data set

In [216]:
df = pd.read_csv('dataset.csv')
print(df.head())


   app_id        app_name                                        review_text  \
0      10  Counter-Strike                                    Ruined my life.   
1      10  Counter-Strike  This will be more of a ''my experience with th...   
2      10  Counter-Strike                      This game saved my virginity.   
3      10  Counter-Strike  • Do you like original games? • Do you like ga...   
4      10  Counter-Strike           Easy to learn, hard to master.             

   review_score  review_votes  
0             1             0  
1             1             1  
2             1             0  
3             1             0  
4             1             1  


In [217]:

# sampling the dataset to decrease run time
sample_size = int(0.01 * len(df))
reduced_sample = df.sample(n=sample_size, random_state=42) 
print(reduced_sample.head())

print(reduced_sample.shape)


         app_id                                   app_name  \
301327    12210  Grand Theft Auto IV: The Complete Edition   
1662500  226320                        Marvel Heroes Omega   
2061157  236450           PAC-MAN Championship Edition DX+   
1171799  218620                                   PAYDAY 2   
1450080  221640                              Super Hexagon   

                                               review_text  review_score  \
301327   Best bowling simulator 2014 10/10 It has good ...             1   
1662500  Marvel characters? Check. Tons of loot? Check....             1   
2061157  This game while its not the original is defina...             1   
1171799  This game ♥♥♥♥ing awesome ,You can be professi...             1   
1450080  If you are high, play this game. 420/420 would...             1   

         review_votes  
301327              1  
1662500             0  
2061157             0  
1171799             0  
1450080             0  
(64171, 5)


## Data Preprocessing and Visualization:

In [218]:
import nltk
import string

# Specify the NLTK data path explicitly
nltk.data.path.append('C:/Users/josh/nltk_data')  # Replace with the actual path to your nltk_data directory

# Download the required NLTK data
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aaron\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aaron\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [219]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string


In [220]:
def preprocess_text_lower(text):
    if isinstance(text, str):
        text = text.lower()
    else:
        cleaned_text = ""

    return text

# tokenize the text
def tokenize_text(text):
    if isinstance(text, str):
        # Tokenization
        tokens = word_tokenize(text)
        cleaned_text = " ".join(tokens)
    else:
        cleaned_text = ""

    return cleaned_text

def remove_punctuation(text):
    if isinstance(text, str):
        # Removing Punctuation
        tokens = word_tokenize(text)
        tokens = [word for word in tokens if word not in string.punctuation]
        cleaned_text = " ".join(tokens)
    else:
        cleaned_text = ""

    return cleaned_text

def remove_stopwords(text):
    if isinstance(text, str):
        # Tokenization
        tokens = word_tokenize(text)

        # Stop Word Removal
        stop_words = set(stopwords.words('english'))
        filtered_tokens = [word for word in tokens if word not in stop_words]

        cleaned_text = " ".join(filtered_tokens)
    else:
        cleaned_text = ""

    return cleaned_text

import re
# handleing things like 10/10
def replace_good_ratings(text):
    pattern = r'(\d+)/(\d+)'

    def replace(match):
        numerator = int(match.group(1))
        denominator = int(match.group(2))

        # Check if the numerator is not 0
        if numerator != 0:
            return 'great'
        else:
            return 'very bad'  # Replace "0/number" with "very bad"

    cleaned_text = re.sub(pattern, replace, text)

    return cleaned_text



In [221]:
# lower case the text
reduced_sample['review_text'] = df['review_text'].apply(preprocess_text_lower)
# 6 seconds

In [222]:
# tokenize the text
reduced_sample['review_text'] = reduced_sample['review_text'].apply(tokenize_text)
# 24 seconds

In [223]:
# remove punctuation
reduced_sample['review_text'] = reduced_sample['review_text'].apply(remove_punctuation)
# 31 seconds

In [224]:
# remove stopwords
reduced_sample['review_text'] = reduced_sample['review_text'].apply(remove_stopwords)
# 50 seconds

In [225]:
# replace good ratings
reduced_sample['review_text'] = reduced_sample['review_text'].apply(replace_good_ratings)


In [226]:
# Print the result (original and cleaned text for the first few rows)
print(reduced_sample.head())

         app_id                                   app_name  \
301327    12210  Grand Theft Auto IV: The Complete Edition   
1662500  226320                        Marvel Heroes Omega   
2061157  236450           PAC-MAN Championship Edition DX+   
1171799  218620                                   PAYDAY 2   
1450080  221640                              Super Hexagon   

                                               review_text  review_score  \
301327    best bowling simulator 2014 great good storyline             1   
1662500  marvel characters check tons loot check tons c...             1   
2061157  game original definately one best renditions p...             1   
1171799  game ♥♥♥♥ing awesome professional heister fun ...             1   
1450080                    high play game great would dank             1   

         review_votes  
301327              1  
1662500             0  
2061157             0  
1171799             0  
1450080             0  


In [227]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix

print(reduced_sample.shape)

# 1. TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # Adjust the number of features as needed
tfidf_matrix = tfidf_vectorizer.fit_transform(reduced_sample['review_text'])
tfidf_matrix = csr_matrix(tfidf_matrix)

# 2. Calculate Sentiment Scores in Batches and Append to DataFrame
batch_size = 1000  # Number of rows to process in each batch
sentiment_scores = []

for start in range(0, len(reduced_sample), batch_size):
    end = min(start + batch_size, len(reduced_sample))
    batch_tfidf_matrix = tfidf_matrix[start:end]
    batch_scores = batch_tfidf_matrix.mean(axis=1)
    sentiment_scores.extend(batch_scores)

# Add the 'sentiment_scores' column to 'reduced_sample' from the TF-IDF scores
reduced_sample['sentiment_scores'] = sentiment_scores


# slopy very bad code... 
def extract_sentiment_score(scores):
    try:
        return float(scores[0][0][0][0])
    except (IndexError, ValueError, TypeError):
        return None

reduced_sample['sentiment_scores'] = reduced_sample['sentiment_scores'].apply(extract_sentiment_score)


# Print the result
print(reduced_sample.head())

(64171, 5)
         app_id                                   app_name  \
301327    12210  Grand Theft Auto IV: The Complete Edition   
1662500  226320                        Marvel Heroes Omega   
2061157  236450           PAC-MAN Championship Edition DX+   
1171799  218620                                   PAYDAY 2   
1450080  221640                              Super Hexagon   

                                               review_text  review_score  \
301327    best bowling simulator 2014 great good storyline             1   
1662500  marvel characters check tons loot check tons c...             1   
2061157  game original definately one best renditions p...             1   
1171799  game ♥♥♥♥ing awesome professional heister fun ...             1   
1450080                    high play game great would dank             1   

         review_votes  sentiment_scores  
301327              1          0.000461  
1662500             0          0.000660  
2061157             0          0.

In [228]:
from afinn import Afinn

# Initialize Afinn sentiment analyzer
afinn = Afinn()

# Function to calculate sentiment score for each word
def calculate_word_sentiment(review):
    words = review.split()
    scores = [afinn.score(word) for word in words]
    return scores

# Calculate sentiment scores for each word in each review
reduced_sample['word_sentiment_scores'] = reduced_sample['review_text'].apply(calculate_word_sentiment)

# Print the result
print(reduced_sample.head())


         app_id                                   app_name  \
301327    12210  Grand Theft Auto IV: The Complete Edition   
1662500  226320                        Marvel Heroes Omega   
2061157  236450           PAC-MAN Championship Edition DX+   
1171799  218620                                   PAYDAY 2   
1450080  221640                              Super Hexagon   

                                               review_text  review_score  \
301327    best bowling simulator 2014 great good storyline             1   
1662500  marvel characters check tons loot check tons c...             1   
2061157  game original definately one best renditions p...             1   
1171799  game ♥♥♥♥ing awesome professional heister fun ...             1   
1450080                    high play game great would dank             1   

         review_votes  sentiment_scores  \
301327              1          0.000461   
1662500             0          0.000660   
2061157             0          0.000735  

## Sentiment Analysis Model

In [234]:
from sklearn.base import BaseEstimator, TransformerMixin

class TextSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]


In [250]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

# Combine review_text and sentiment scores
combined_features = reduced_sample['review_text'].astype(str) + ' ' + reduced_sample['word_sentiment_scores'].apply(lambda x: ' '.join(map(str, x)))

# TF-IDF vectorization for text
tfidf_vectorizer = TfidfVectorizer()
text_features = tfidf_vectorizer.fit_transform(combined_features)

# Convert sentiment scores to a numpy array
sentiment_scores = reduced_sample['word_sentiment_scores'].apply(lambda x: sum(x) / len(x) if len(x) > 0 else 0).values.reshape(-1, 1)

# Concatenate text and sentiment features
X = hstack((text_features, sentiment_scores))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
clf = LogisticRegression(random_state=42, max_iter=1000)
clf.fit(X_train, y_train)

# Evaluate the model
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)


Accuracy: 0.8668484612388001


## Make Predictions

In [249]:
# Loop through a few examples from the test set
num_examples_to_print = 5  # Change this to the number of examples you want to print

for i in range(num_examples_to_print):
    review_text = reduced_sample['review_text'].iloc[i]  # Get the review text
    review_score = y_test.iloc[i]  # Get the corresponding review score from the test set
    predicted_score = y_pred[i]  # Get the predicted review score
    
    print(f"Review Text: {review_text}")
    print(f"Actual Review Score: {review_score}")
    print(f"Predicted Review Score: {predicted_score}")
    print("\n")


Review Text: best bowling simulator 2014 great good storyline
Actual Review Score: 1
Predicted Review Score: 1


Review Text: marvel characters check tons loot check tons characters/classes check interesting events check lfg gtfo solo marvel fan diablo fan good marvel fan still better d3 2013
Actual Review Score: -1
Predicted Review Score: 1


Review Text: game original definately one best renditions pac-man ever totally recommend even though really came 2007 2013 steam -_-
Actual Review Score: 1
Predicted Review Score: 1


Review Text: game ♥♥♥♥ing awesome professional heister fun play friend fun add workshop mod
Actual Review Score: -1
Predicted Review Score: 1


Review Text: high play game great would dank
Actual Review Score: 1
Predicted Review Score: 1




In [283]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Define the function to predict sentiment
def predict_review_sentiment(review_text, clf, vectorizer, afinn):
    # Preprocess the review text (you should use the same preprocessing steps you used on the training data)
    # ...

    # Calculate sentiment scores
    sentiment_scores = [afinn.score(word) for word in review_text.split()]
    mean_sentiment_score = sum(sentiment_scores) / len(sentiment_scores) if len(sentiment_scores) > 0 else 0

    # Transform the review text using the TF-IDF vectorizer
    text_features = vectorizer.transform([review_text])
    
    # Combine text and sentiment features
    combined_features = hstack((text_features, mean_sentiment_score))

    # Make a prediction
    prediction = clf.predict(combined_features)
    return 'Positive' if prediction[0] == 1 else 'Negative'

# Test the model on sample review texts
sample_reviews = [
    "This game is amazing!",
    "Worst game ever.",
    "I love this game, it's the best!",
    "I hate this game, it's terrible.",
]

# Loop through sample reviews
for review in sample_reviews:
    sentiment = predict_review_sentiment(review, clf, tfidf_vectorizer, afinn)
    print(f"Review: {review}\nSentiment: {sentiment}\n")


Review: This game is amazing!
Sentiment: Positive

Review: Worst game ever.
Sentiment: Negative

Review: I love this game, it's the best!
Sentiment: Positive

Review: I hate this game, it's terrible.
Sentiment: Negative



In [ ]:
def predict_review_sentiment(review_text, clf, vectorizer):
    # Preprocess the review text (you should use the same preprocessing steps you used on the training data)
    # ...

    # Calculate sentiment scores
    sentiment_scores = [afinn.score(word) for word in review_text.split()]
    mean_sentiment_score = sum(sentiment_scores) / len(sentiment_scores) if len(sentiment_scores) > 0 else 0

    # Transform the review text using the TF-IDF vectorizer
    tfidf_vector = vectorizer.transform([review_text])
    
    # Make a prediction
    prediction = clf.predict([[mean_sentiment_score]])
    return 'Positive' if prediction[0] == 1 else 'Negative'

# Test the model on sample review texts
sample_reviews = [
    "This game is amazing!",
    "Worst game ever.",
    "I love this game, it's the best!",
    "I hate this game, it's terrible.",
]
for review in sample_reviews:
    sentiment = predict_review_sentiment(review, clf, tfidf_vectorizer)
    print(f"Review: {review}\nSentiment: {sentiment}\n")


Review: This game is amazing!
Sentiment: Positive

Review: Worst game ever.
Sentiment: Negative

Review: I love this game, it's the best!
Sentiment: Positive

Review: I hate this game, it's terrible.
Sentiment: Negative



## Evaluate the Model

## Summary

In [ ]:
nltk.download('vader_lexicon')
nltk.data.path.append('C:/Users/josh/nltk_data')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\aaron\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
